ASR

In [1]:
# Needs Uni enviornment

In [2]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [3]:
# pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [4]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [5]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']

# searchTerm1 = ['a']
# searchTerm2 = ['the']

tc = 0
noResultsList = []

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)

baseUrl = 'https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22'
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"%22+AND+%22"+ item2+"%22&Ppub=&startPage=&SeriesKey=tasr20"
        print(query)
        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        tc = tc + number
        
        if number == 0:
            noResultsList.append((item1 +","+item2))
        
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
#         print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize = 2

#         for currentPage in range(0,2):
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
                        
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)
#             print(allLinks)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
#                 print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
            print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1
            print("row: ",row)


['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Sustainable+building%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22BIM%22+AND+%22Sustainable+buildings%22&Ppub=&startPage=&SeriesKey=t

https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Water%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Waste%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Thermal+comfort%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Electricity%22&Ppub=&startPage=&SeriesKey=tasr20
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Electricity%22&Ppub=&startPage=&SeriesKey=tasr20
Number of sections:  8
row:  12
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Building+information+model%22+AND+%22Solar%22&Ppub=&startPage=&SeriesKey=tasr20
In page:  0
Search

https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Wind+power%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Green+infrastructure%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Smart+construction%22+AND+%22Life+cycle+assessment%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Green+Building%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Green+Buildings%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engineering%22+AND+%22Sustainable+building%22&Ppub=&startPage=&SeriesKey=tasr20
https://www.tandfonline.com/action/doSearch?field1=Abstract&text1=%22Digital+engi

In [6]:
noResultsList

['BIM,Green+Building',
 'BIM,Green+Buildings',
 'BIM,Sustainable+building',
 'BIM,Sustainable+buildings',
 'BIM,Environmentally+friendly+design',
 'BIM,Sustainable+development',
 'BIM,Water',
 'BIM,Waste',
 'BIM,Thermal+comfort',
 'BIM,Wind+power',
 'BIM,Green+infrastructure',
 'BIM,Life+cycle+assessment',
 'Building+Information+Modelling,Green+Building',
 'Building+Information+Modelling,Green+Buildings',
 'Building+Information+Modelling,Sustainable+building',
 'Building+Information+Modelling,Sustainable+buildings',
 'Building+Information+Modelling,Environmentally+friendly+design',
 'Building+Information+Modelling,Sustainable+development',
 'Building+Information+Modelling,Water',
 'Building+Information+Modelling,Waste',
 'Building+Information+Modelling,Thermal+comfort',
 'Building+Information+Modelling,Wind+power',
 'Building+Information+Modelling,Green+infrastructure',
 'Building+Information+Modelling,Life+cycle+assessment',
 'Building+information+model,Green+Building',
 'Building+inf

In [7]:
df

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Sustainability,https://www.tandfonline.com/doi/full/10.1080/0...,BIM for sustainable project delivery: review p...,ABSTRACT,The evolution of Building Information Modellin...,1. Introduction,Building Information Modelling (BIM) is define...,2. Scope of the research,"As can be seen from section one, and summarize...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Energy,https://www.tandfonline.com/doi/full/10.1080/0...,Closing the gap in building performance: learn...,Abstract,It is clear that the current industry process ...,1. Introduction: a selection of current buildi...,"In the partner paper by the same authors, it w...",2. Comparison with BIM benchmark industries’ p...,The BIM initiative in the UK was promoted on t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Energy,https://www.tandfonline.com/doi/full/10.1080/0...,Linking design and simulation using non-manifo...,Abstract,The aim of this paper is to propose a differen...,Introduction,"Traditionally, building information modelling ...",BPS in the early design stages,It is generally agreed that energy analysis is...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Energy,https://www.tandfonline.com/doi/full/10.1080/0...,Are current design processes and policies deli...,Abstract,Gaps between intended and actual performance w...,"1. Introduction, aims and approach",Many policy- and industry-driven initiatives w...,2. Gaps between intended and actual building p...,Researchers globally are finding that building...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Electricity,https://www.tandfonline.com/doi/full/10.1080/0...,Streamlining sustainable design in building in...,AbstractFormulae display:?Mathematical formula...,The current BIM-based PV system design process...,1. Introduction,"In 2018, the US consumed 101.3 quadrillions BT...",2. BIM-based PV system design and management,"To realize the goal of PV system design, const...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BIM,Solar,https://www.tandfonline.com/doi/full/10.1080/0...,Streamlining sustainable design in building in...,AbstractFormulae display:?Mathematical formula...,The current BIM-based PV system design process...,1. Introduction,"In 2018, the US consumed 101.3 quadrillions BT...",2. BIM-based PV system design and management,"To realize the goal of PV system design, const...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Building+Information+Modelling,Sustainability,https://www.tandfonline.com/doi/full/10.1080/0...,BIM for sustainable project delivery: review p...,ABSTRACT,The evolution of Building Information Modellin...,1. Introduction,Building Information Modelling (BIM) is define...,2. Scope of the research,"As can be seen from section one, and summarize...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Building+Information+Modelling,Energy,https://www.tandfonline.com/doi/full/10.1080/0...,Linking design and simulation using non-manifo...,Abstract,The aim of this paper is to propose a differen...,Introduction,"Traditionally, building information modelling ...",BPS in the early design stages,It is generally agreed that energy analysis is...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Building+Information+Modelling,Energy,https://www.tandfonline.com/doi/full/10.1080/0...,Closing the gap in building performance: learn...,Abstract,It is clear that the current industry process ...,1. Introduction: a selection of current buildi...,"In the partner paper by the same authors, it w...",2. Comparison with BIM benchmark industries’ p...,The BIM initiative in the UK was promoted on t...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Building+Information+Modelling,Electricity,https://www.tandfonline.com/doi/full/10.1080/0...,Streamlining sustainable design in building in...,Abst

In [8]:
df.to_csv('kgRawData/KG-ASR.csv')

In [1]:
journalName = 'asr'

import pandas as pd
df = pd.read_csv('kgRawData/KG-'+ journalName +'.csv')
df = df.where(pd.notnull(df), None)

import re
absRows = []
rowsFound = []
for i in range(len(df['url'])):
    text = df['url'][i]
    pattern = '/abs/'
    if re.search(pattern, text):
        absRows.append(i)
    else:
        rowsFound.append(i)
        
rowsFoundDf = pd.DataFrame()
rowsFoundDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

rowsFoundDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
rowsFoundDf = rowsFoundDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    rowsFoundDf[section] = ''
    rowsFoundDf[sectionData] = ''

    
absRowsDf = pd.DataFrame()
absRowsDf.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

absRowsDf = pd.read_csv("blankFile.csv", names= columnNames , header = None)
absRowsDf = absRowsDf.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    absRowsDf[section] = ''
    absRowsDf[sectionData] = ''
    
    
for entry in range(len(rowsFound)):
    rowsFoundDf.loc[entry] = df.loc[rowsFound[entry]]

for entry in range(len(absRows)):
    absRowsDf.loc[entry] = df.loc[absRows[entry]]

rowsFoundDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-full.csv')
absRowsDf.to_csv('kgRawData/tandf/kg-'+ journalName +'-abs.csv')

print(len(absRowsDf))
print(len(rowsFoundDf))

0
13
